In [13]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.edge.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [28]:


# # Đường dẫn tới msedgedriver.exe
# service=Service(exucutable_path=r"C:\Users\DO TUAN MINH\Desktop\edgedriver_win64\msedgedriver.exe")

# # Tạo trình duyệt Edge
# driver = webdriver.Edge(service=service)

options = Options()
options.add_argument(r"user-data-dir=C:\Users\DO TUAN MINH\AppData\Local\Microsoft\Edge\User Data\Default")
options.add_argument("--disable-blink-features=AutomationControlled")
# nếu profile là Default thì không cần thêm profile-dir

service=Service(exucutable_path=r"C:\Users\DO TUAN MINH\Desktop\edgedriver_win64\msedgedriver.exe")
driver = webdriver.Edge(service=service, options=options)

# Truy cập trang login
# driver.get("https://fap.fpt.edu.vn/Default.aspx")  # sửa lại URL thực tế
driver.get("https://flm.fpt.edu.vn/")
time.sleep(4)

# # Click vào nút "Đăng nhập bằng FEID"
# login_button = driver.find_element(By.ID, "ctl00_mainContent_btnloginFeId")
# login_button.click()
# time.sleep(5)

# # Bước 4: Click nút "Email fpt.edu.vn hoặc Gmail"
# gmail_btn = driver.find_element(By.XPATH, "//a[contains(@href, 'External/Challenge') and contains(@href, 'scheme=Google')]")
# gmail_btn.click()

# Tìm nút và nhấn
login_button = driver.find_element(By.XPATH, '//a[@href="/Home/LoginWithFEID"]')
login_button.click()
time.sleep(3)
gmail_btn = driver.find_element(By.XPATH, "//a[contains(@href, 'External/Challenge') and contains(@href, 'scheme=Google')]")
gmail_btn.click()
# Chờ bạn login Gmail (lần đầu)
# input("Sau khi đăng nhập Gmail xong, nhấn Enter để tiếp tục...")

# Lấy HTML của trang sau login
# html = driver.page_source
# print(html)
time.sleep(1)
curriculum= driver.find_element(By.XPATH, '//*[@id="panelStudentFeatures"]/div/div/div/div/div/div/div/div/a[1]')
curriculum.click()
time.sleep(2)
input_box = driver.find_element(By.ID, "txtKeyword")
input_box.send_keys("AI")
time.sleep(1)

btn_search=driver.find_element(By.ID,'btnSearch')
btn_search.click()
time.sleep(3)




# Lấy chương trình học đầu tiên
first_program = driver.find_element(By.XPATH, '//a[contains(@href, "CurriculumDetails.aspx")]')
program_name = first_program.text
program_href = first_program.get_attribute('href')

# Truy cập trang chi tiết chương trình học
driver.get(program_href)

# Chờ bảng môn học hiện ra
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//table[contains(@class, "table")]'))
)


# Tìm bảng
table = driver.find_element(By.ID, "gvSubs")  # hoặc By.ID nếu có ID riêng

# Lấy tiêu đề cột
headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]

# Thêm cột mới cho link
headers.append("SyllabusLink")

# Lấy từng hàng dữ liệu
data = []
rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Bỏ dòng tiêu đề

for i in range(len(rows)):
    # Reload bảng mỗi lần vì driver.back() làm mất DOM cũ
    table = driver.find_element(By.ID, "gvSubs")
    rows = table.find_elements(By.TAG_NAME, "tr")[1:]
    row = rows[i]
    
    cols = row.find_elements(By.TAG_NAME, "td")
    subject_name = cols[0].text.strip()

    row_data = [col.text.strip() for col in cols]
    raw_html = ""

    # Nếu có thẻ <a>, thì có thể lấy HTML
    # Trong vòng lặp mỗi row
    a_tags = cols[1].find_elements(By.TAG_NAME, "a")
    subject_link = ""
    raw_html = ""

    if a_tags:
        subject_link = a_tags[0].get_attribute("href")

        # Nếu KHÔNG phải COM hoặc ELE → vào link và lấy HTML
        if not ("COM" in subject_name or "ELE" in subject_name):
            driver.get(subject_link)
            time.sleep(1)
            raw_html = driver.page_source
            driver.back()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "gvSubs"))
            )

    # Thêm link và raw_html vào row
    row_data.append(subject_link)
    row_data.append(raw_html)
    data.append(row_data)

# Lấy tiêu đề cột + thêm RawHTML
table = driver.find_element(By.ID, "gvSubs")
headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
headers.extend(["SubjectLink", "RawHTML"])



# Tạo DataFrame và lưu
df = pd.DataFrame(data, columns=headers)
# df.to_csv("subjects.csv", index=False, encoding="utf-8-sig")
print(df.head())
# print("✅ Đã lưu xong subjects.csv")
driver.quit()


  SubjectCode                                       Subject Name Semester  \
0      OTP101  Orientation and General Training Program_Định ...        0   
1         PEN             Preparation English_Tiếng Anh chuẩn bị        0   
2   PHE_COM*1           Physical Education 1_Giáo dục thể chất 1        0   
3     TMI_ELE  Traditional musical instrument_Nhạc cụ truyền ...        0   
4      CSI106  Introduction to Computer Science_Nhập môn khoa...        1   

  NoCredit PreRequisite                                        SubjectLink  \
0        0         None  https://flm.fpt.edu.vn/gui/role/student/Syllab...   
1        0               https://flm.fpt.edu.vn/gui/role/student/Syllab...   
2        2               https://flm.fpt.edu.vn/gui/role/student/Syllab...   
3        3               https://flm.fpt.edu.vn/gui/role/student/Syllab...   
4        3               https://flm.fpt.edu.vn/gui/role/student/Syllab...   

                                             RawHTML  
0  <html lang

In [29]:
df.to_csv("flm_subjects.csv", index=False, encoding="utf-8-sig")

In [32]:
df.columns

Index(['SubjectCode', 'Subject Name', 'Semester', 'NoCredit', 'PreRequisite',
       'SubjectLink', 'RawHTML'],
      dtype='object')

In [62]:


# # Đường dẫn tới msedgedriver.exe
# service=Service(exucutable_path=r"C:\Users\DO TUAN MINH\Desktop\edgedriver_win64\msedgedriver.exe")

# # Tạo trình duyệt Edge
# driver = webdriver.Edge(service=service)

options = Options()
options.add_argument(r"user-data-dir=C:\Users\DO TUAN MINH\AppData\Local\Microsoft\Edge\User Data\Default")
options.add_argument("--disable-blink-features=AutomationControlled")
# nếu profile là Default thì không cần thêm profile-dir

service=Service(exucutable_path=r"C:\Users\DO TUAN MINH\Desktop\edgedriver_win64\msedgedriver.exe")
driver = webdriver.Edge(service=service, options=options)

# Truy cập trang login
# driver.get("https://fap.fpt.edu.vn/Default.aspx")  # sửa lại URL thực tế
driver.get("https://flm.fpt.edu.vn/")
time.sleep(4)

# # Click vào nút "Đăng nhập bằng FEID"
# login_button = driver.find_element(By.ID, "ctl00_mainContent_btnloginFeId")
# login_button.click()
# time.sleep(5)

# # Bước 4: Click nút "Email fpt.edu.vn hoặc Gmail"
# gmail_btn = driver.find_element(By.XPATH, "//a[contains(@href, 'External/Challenge') and contains(@href, 'scheme=Google')]")
# gmail_btn.click()

# Tìm nút và nhấn
login_button = driver.find_element(By.XPATH, '//a[@href="/Home/LoginWithFEID"]')
login_button.click()
time.sleep(3)
gmail_btn = driver.find_element(By.XPATH, "//a[contains(@href, 'External/Challenge') and contains(@href, 'scheme=Google')]")
gmail_btn.click()
# Chờ bạn login Gmail (lần đầu)
# input("Sau khi đăng nhập Gmail xong, nhấn Enter để tiếp tục...")

# Lấy HTML của trang sau login
# html = driver.page_source
# print(html)
time.sleep(1)
curriculum= driver.find_element(By.XPATH, '//*[@id="panelStudentFeatures"]/div/div/div/div/div/div/div/div/a[1]')
curriculum.click()
time.sleep(2)
input_box = driver.find_element(By.ID, "txtKeyword")
input_box.send_keys("AI")
time.sleep(1)

btn_search=driver.find_element(By.ID,'btnSearch')
btn_search.click()
time.sleep(3)




# Lấy chương trình học đầu tiên
first_program = driver.find_element(By.XPATH, '//a[contains(@href, "CurriculumDetails.aspx")]')
program_name = first_program.text
program_href = first_program.get_attribute('href')

# Truy cập trang chi tiết chương trình học
driver.get(program_href)

# Chờ bảng môn học hiện ra
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//table[contains(@class, "table")]'))
)


# Tìm bảng
table = driver.find_element(By.ID, "gvSubs")  # hoặc By.ID nếu có ID riêng

# Lấy tiêu đề cột
headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]

# Thêm cột mới cho link
headers.append("SyllabusLink")

# Lấy từng hàng dữ liệu
data = []
rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Bỏ dòng tiêu đề

for i in range(len(rows)):
    # Reload bảng mỗi lần vì driver.back() làm mất DOM cũ
    table = driver.find_element(By.ID, "gvSubs")
    rows = table.find_elements(By.TAG_NAME, "tr")[1:]
    row = rows[i]
    
    cols = row.find_elements(By.TAG_NAME, "td")
    subject_name = cols[0].text.strip()

    row_data = [col.text.strip() for col in cols]
    raw_html = ""

    # Nếu có thẻ <a>, thì có thể lấy HTML
    # Trong vòng lặp mỗi row
    a_tags = cols[1].find_elements(By.TAG_NAME, "a")
    subject_link = ""
    raw_html = ""
    parent_subject = ""
    belong_to_combo = ""
    if a_tags:
        subject_link = a_tags[0].get_attribute("href")

        # Nếu KHÔNG phải COM hoặc ELE → vào link và lấy HTML
        # Xử lý riêng cho COM
        if "COM" in subject_name:
            row_data.append(subject_link)
            row_data.append(raw_html)
            row_data.extend([parent_subject,belong_to_combo])
            data.append(row_data)

            parent_subject = subject_name
            

            if subject_link:
                driver.get(subject_link)
                time.sleep(1)

                # Lấy danh sách combo (giả định combo nằm trong div.auto-style11 > h3, thay đổi theo thực tế)
                combos = driver.find_elements(By.CSS_SELECTOR, "div.auto-style11 > h3")
                sub_list = []
                for combo in combos:
                    combo_name = combo.text.strip()

                    # Lấy bảng môn con
                    table_element = combo.find_element(By.XPATH, "following-sibling::table[1]")
                    sub_rows = table_element.find_elements(By.TAG_NAME, "tr")[1:]
                    print(combo_name)
                    # Tạo list lưu các sub môn tạm trước khi xử lý link
                    
                    
                    for sub_row in sub_rows:
                        sub_cols = sub_row.find_elements(By.TAG_NAME, "td")
                        sub_data = [col.text.strip() for col in sub_cols[:2]]

                        # Lấy các thông tin semester, no_credit, pre_requisite
                        sub_semester = row_data[2]
                        sub_no_credit = row_data[3]
                        sub_pre_requisite = row_data[4]


                        a_sub = sub_cols[1].find_elements(By.TAG_NAME, "a")
                        sub_link = a_sub[0].get_attribute("href") if a_sub else ""

                        # Lưu tạm thông tin chứ chưa truy cập link
                        sub_list.append((sub_data, sub_semester, sub_no_credit, sub_pre_requisite, sub_link,"","",combo_name))
                    
                # Xử lý link từng sub môn một ngoài vòng lặp combos để tránh stale element
                for (sub_data, sub_semester, sub_no_credit, sub_pre_requisite, sub_link,_,_,combo_name) in sub_list:
                    raw_html_sub = ""
                    if sub_link:
                        driver.get(sub_link)
                        time.sleep(1)
                        raw_html_sub = driver.page_source
                        driver.back()
                        time.sleep(1)
                        # Nên chờ element tải lại nếu cần, ví dụ:
                        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.auto-style11 > h3")))
                    
                    # Thêm vào data cuối cùng
                    data.append(sub_data + [sub_semester, sub_no_credit, sub_pre_requisite, sub_link, raw_html_sub, parent_subject, combo_name])

                # Lấy raw_html của trang combo tổng thể nếu muốn
                # raw_html = driver.page_source

                driver.back()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "gvSubs")))

        # Xử lý riêng cho ELE
        elif "ELE" in subject_name:
            row_data.append(subject_link)
            row_data.append(raw_html)
            row_data.extend([parent_subject,belong_to_combo])
            data.append(row_data)
            parent_subject = subject_name
            belong_to_combo = ""  # ELE không có combo, để rỗng hoặc tùy chỉnh

            if subject_link:
                driver.get(subject_link)
                time.sleep(1)

                # Giả định môn con liệt kê trong bảng giống bảng chính, lấy tất cả dòng
                sub_table = driver.find_element(By.CSS_SELECTOR, "table[style*='margin-bottom']")  # hoặc id khác nếu trang khác
                sub_rows = sub_table.find_elements(By.TAG_NAME, "tr")[1:]

                # Thu thập data + link trước
                sub_info_list = []
                for sub_row in sub_rows:
                    sub_cols = sub_row.find_elements(By.TAG_NAME, "td")
                    sub_data = [col.text.strip() for col in sub_cols[:2]]
                    
                    sub_semester = row_data[2]
                    sub_no_credit = row_data[3]
                    sub_pre_requisite = row_data[4]

                    a_sub = sub_cols[1].find_elements(By.TAG_NAME, "a")
                    sub_link = a_sub[0].get_attribute("href") if a_sub else ""

                    sub_info_list.append((sub_data, sub_semester, sub_no_credit, sub_pre_requisite, sub_link))

                # Duyệt link riêng để tránh stale element
                for sub_data, sub_semester, sub_no_credit, sub_pre_requisite, sub_link in sub_info_list:
                    raw_html_sub = ""
                    if sub_link:
                        driver.get(sub_link)
                        time.sleep(1)
                        raw_html_sub = driver.page_source
                        driver.back()
                        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "gvSubs")))
                        time.sleep(1)

                    data.append(sub_data + [sub_semester, sub_no_credit, sub_pre_requisite, sub_link, raw_html_sub, parent_subject, belong_to_combo])

                raw_html = driver.page_source
                driver.back()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "gvSubs")))

        if not ("COM" in subject_name or "ELE" in subject_name):
            driver.get(subject_link)
            time.sleep(1)
            raw_html = driver.page_source
            driver.back()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "gvSubs"))
            )

            # Thêm link và raw_html vào row
            row_data.append(subject_link)
            row_data.append(raw_html)
            row_data.extend([parent_subject,belong_to_combo])
            data.append(row_data)

# Lấy tiêu đề cột + thêm RawHTML
table = driver.find_element(By.ID, "gvSubs")
headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
headers.extend(["SubjectLink", "RawHTML", "ParentSubject", "BelongToCombo"])



# Tạo DataFrame và lưu
df = pd.DataFrame(data, columns=headers)
# df.to_csv("subjects.csv", index=False, encoding="utf-8-sig")
print(df.head())
# print("✅ Đã lưu xong subjects.csv")
driver.quit()


PHE_COM1: Vovinam BIT_AI_K17C(FNO)
PHE_COM2: Chess_Cờ vua BIT_AI_K17C(FNO)
PHE_COM1: Vovinam BIT_AI_K17C(FNO)
PHE_COM2: Chess_Cờ vua BIT_AI_K17C(FNO)
PHE_COM1: Vovinam BIT_AI_K17C(FNO)
PHE_COM2: Chess_Cờ vua BIT_AI_K17C(FNO)
AI17_COM1: Topic on Applied Data Science_Chủ đề Khoa học dữ liệu ứng dụng BIT_AI_K17C(FNO)
AI17_COM3: Topic on AI application_Chủ đề Ứng dụng TTNT BIT_AI_K17C(FNO)
AI17_COM2.1: Topic on AI Applied research_Chủ đề Định hướng nghiên cứu ứng dụng TTNT
AI17_COM1: Topic on Applied Data Science_Chủ đề Khoa học dữ liệu ứng dụng BIT_AI_K17C(FNO)
AI17_COM3: Topic on AI application_Chủ đề Ứng dụng TTNT BIT_AI_K17C(FNO)
AI17_COM2.1: Topic on AI Applied research_Chủ đề Định hướng nghiên cứu ứng dụng TTNT
AI17_COM1: Topic on Applied Data Science_Chủ đề Khoa học dữ liệu ứng dụng BIT_AI_K17C(FNO)
AI17_COM3: Topic on AI application_Chủ đề Ứng dụng TTNT BIT_AI_K17C(FNO)
AI17_COM1: Topic on Applied Data Science_Chủ đề Khoa học dữ liệu ứng dụng BIT_AI_K17C(FNO)
AI17_COM3: Topic on AI

In [64]:
df.to_csv("FULL_subjects.csv", index=False, encoding="utf-8-sig")